In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

--2025-10-13 16:00:01--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.005s  

2025-10-13 16:00:02 (16.1 MB/s) - ‘course_lead_scoring.csv’ saved [80876/80876]



In [19]:
import pandas as pd

df = pd.read_csv('course_lead_scoring.csv')
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [10]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [24]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
categorical_cols = df.dtypes[df.dtypes == 'object'].index 

for c in categorical_cols:
    df[c] = df[c].str.lower().str.replace(' ','_')
    
df.head().T

,0,1,2,3,4
lead_source,paid_ads,social_media,events,paid_ads,referral
industry,NaN,retail,healthcare,retail,education
number_of_courses_viewed,1,1,5,2,3
annual_income,79450.0,46992.0,78796.0,83843.0,85012.0
employment_status,unemployed,employed,unemployed,NaN,self_employed
location,south_america,south_america,australia,australia,europe
interaction_count,4,1,3,1,3
lead_score,0.94,0.8,0.69,0.87,0.62
converted,1,0,1,0,1


In [27]:
df.number_of_courses_viewed = pd.to_numeric(df.number_of_courses_viewed, errors='coerce')
df.annual_income = pd.to_numeric(df.annual_income, errors='coerce')
df.interaction_count = pd.to_numeric(df.interaction_count, errors='coerce')
df.lead_score = pd.to_numeric(df.lead_score, errors='coerce')

In [57]:
df.number_of_courses_viewed = df.number_of_courses_viewed.fillna(0)
df.annual_income = df.annual_income.fillna(0)
df.interaction_count = df.interaction_count.fillna(0)
df.lead_score = df.lead_score.fillna(0)

In [58]:
for c in categorical_cols:
    df[c] = df[c].fillna('NA')

In [30]:
df.head().T

,0,1,2,3,4
lead_source,paid_ads,social_media,events,paid_ads,referral
industry,NA,retail,healthcare,retail,education
number_of_courses_viewed,1,1,5,2,3
annual_income,79450.0,46992.0,78796.0,83843.0,85012.0
employment_status,unemployed,employed,unemployed,NA,self_employed
location,south_america,south_america,australia,australia,europe
interaction_count,4,1,3,1,3
lead_score,0.94,0.8,0.69,0.87,0.62
converted,1,0,1,0,1


In [59]:
df['industry'].value_counts()


industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

In [69]:
numerical = ['number_of_courses_viewed','annual_income','interaction_count','lead_score']
df[numerical].corr()


,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


In [ ]:
from sklearn.model_selection import train_test_split

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split (df_full_train, test_size=0.25, random_state=42)

len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [33]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [35]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

In [36]:
df_full_train = df_full_train.reset_index(drop=True)
df_full_train.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [62]:
from sklearn.metrics import mutual_info_score

for c in categorical_cols:
    print(c, mutual_info_score(y_train, df_train[c]))

lead_source 0.024803322681970594
industry 0.006160554077226654
employment_status 0.016344665458739693
location 0.0014525301016858547


In [98]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=False)

categorical = categorical_cols.tolist()

train_dict = df_train[categorical + numerical].to_dict(orient='records')
print(train_dict)
X_train = dv.fit_transform(train_dict)

print(X_train)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

print(X_val)

[{'lead_source': 'events', 'industry': 'manufacturing', 'employment_status': 'unemployed', 'location': 'europe', 'number_of_courses_viewed': 2, 'annual_income': 95543.0, 'interaction_count': 3, 'lead_score': 0.78}, {'lead_source': 'referral', 'industry': 'NA', 'employment_status': 'student', 'location': 'south_america', 'number_of_courses_viewed': 1, 'annual_income': 54924.0, 'interaction_count': 6, 'lead_score': 0.39}, {'lead_source': 'organic_search', 'industry': 'healthcare', 'employment_status': 'unemployed', 'location': 'europe', 'number_of_courses_viewed': 2, 'annual_income': 77352.0, 'interaction_count': 2, 'lead_score': 0.22}, {'lead_source': 'paid_ads', 'industry': 'other', 'employment_status': 'employed', 'location': 'south_america', 'number_of_courses_viewed': 2, 'annual_income': 34600.0, 'interaction_count': 2, 'lead_score': 0.31}, {'lead_source': 'paid_ads', 'industry': 'education', 'employment_status': 'unemployed', 'location': 'south_america', 'number_of_courses_viewed':

In [95]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict_proba(X_val)[:, 1]

churn_decision = (y_pred >= 0.5)

(churn_decision == y_val).mean().round(2)



np.float64(0.7)